En este notebook vamos a usar [esta libreria de ensamblado](https://github.com/TorchEnsemble-Community/Ensemble-Pytorch) para intentar mejorar el acc de neustro modelo

## Pequeño test antes de cargar el modelo grande

Este ejemplo está extraido de la siguiente guía: https://ensemble-pytorch.readthedocs.io/en/latest/quick_start.html#set-the-logger

In [1]:
import torch.nn as nn
from torch.nn import functional as F

### Define Your Base Estimator

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(147456, 128)
        self.linear2 = nn.Linear(128, 128)
        self.linear3 = nn.Linear(128, 10)

    def forward(self, data):
        data = data.view(data.size(0), -1)  # flatten
        output = F.relu(self.linear1(data))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        return output


### Set the Logger

In [3]:
from torchensemble.utils.logging import set_logger

logger = set_logger("classification_mnist_mlp")


# Descomentar esto para usar tensorboard.
# logger = set_logger("classification_mnist_mlp", use_tb_logger=True)
# Usar "tensorboard --logdir=logs/" para ver los logs en la consola (https://www.tensorflow.org/tensorboard)
# Más info de tensorboard con PyTorch aquí: https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html


Log will be saved in 'd:\Proyectos\TFG\Sign2Text\Project\src\ensemble\logs'.
Start logging into file d:\Proyectos\TFG\Sign2Text\Project\src\ensemble\logs\classification_mnist_mlp-2022_04_01_19_07.log...


### Choose the Ensemble

In [4]:
from torchensemble import VotingClassifier

model = VotingClassifier(
    estimator=MLP,
    n_estimators=10,
    cuda=True,
)

### Set the Criterion

In [5]:
criterion = nn.CrossEntropyLoss()
model.set_criterion(criterion)

### Set the Optimizer ans Scheduler

In [6]:
model.set_optimizer(
    "Adam",  # parameter optimizer
    lr=1e-3,  # learning rate of the optimizer
    weight_decay=5e-4,
)  # weight decay of the optimizer

# model.set_scheduler(
#     "",  # scheduler
#     mode="min",  # mode of the scheduler
#     factor=0.5,  # factor of the scheduler
#     patience=5,  # patience of the scheduler
# )  # verbose of the scheduler


### Get loader for trainning and test

In [7]:
import sys
sys.path.insert(1, "../")

from nets.ResNet.utils.loader import get_dataset, split_dataset
from nets.ResNet.config.torch_config import get_transform


In [8]:
transform = get_transform(image_size=128, width_multiplier=3)
data_path = "../../data/WLASL/frames_5"


In [9]:
dataset, classes = get_dataset(data_path, transform)
train_loader, test_loader = split_dataset(dataset, train_split=0.7, batch_size=16)


### Train and Evaluate

In [10]:
# Model export src.
src = "../../models/example_ensemble"

In [11]:
# Training
model.fit(
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=20,
    save_model=True,
    save_dir=src,  # training data
)  # the number of training epochs


Estimator: 000 | Epoch: 000 | Batch: 000 | Loss: 2.28855 | Correct: 1/16
Estimator: 001 | Epoch: 000 | Batch: 000 | Loss: 2.27660 | Correct: 6/16
Estimator: 002 | Epoch: 000 | Batch: 000 | Loss: 2.30829 | Correct: 2/16
Estimator: 003 | Epoch: 000 | Batch: 000 | Loss: 2.26127 | Correct: 8/16
Estimator: 004 | Epoch: 000 | Batch: 000 | Loss: 2.30542 | Correct: 5/16
Estimator: 005 | Epoch: 000 | Batch: 000 | Loss: 2.25813 | Correct: 1/16
Estimator: 006 | Epoch: 000 | Batch: 000 | Loss: 2.32737 | Correct: 0/16
Estimator: 007 | Epoch: 000 | Batch: 000 | Loss: 2.27499 | Correct: 3/16
Estimator: 008 | Epoch: 000 | Batch: 000 | Loss: 2.29777 | Correct: 4/16
Estimator: 009 | Epoch: 000 | Batch: 000 | Loss: 2.37391 | Correct: 2/16


2022-04-01 19:07:31,656 - INFO: Saving the model to `../../models/example_ensemble\VotingClassifier_MLP_10_ckpt.pth`
2022-04-01 19:07:34,134 - INFO: Epoch: 000 | Validation Acc: 33.333 % | Historical Best: 33.333 %


Estimator: 000 | Epoch: 001 | Batch: 000 | Loss: 3.23469 | Correct: 7/16
Estimator: 001 | Epoch: 001 | Batch: 000 | Loss: 14.69904 | Correct: 5/16
Estimator: 002 | Epoch: 001 | Batch: 000 | Loss: 9.14375 | Correct: 5/16
Estimator: 003 | Epoch: 001 | Batch: 000 | Loss: 17.22717 | Correct: 4/16
Estimator: 004 | Epoch: 001 | Batch: 000 | Loss: 2.91366 | Correct: 4/16
Estimator: 005 | Epoch: 001 | Batch: 000 | Loss: 8.24148 | Correct: 3/16
Estimator: 006 | Epoch: 001 | Batch: 000 | Loss: 5.48035 | Correct: 2/16
Estimator: 007 | Epoch: 001 | Batch: 000 | Loss: 2.07989 | Correct: 7/16
Estimator: 008 | Epoch: 001 | Batch: 000 | Loss: 5.31751 | Correct: 4/16
Estimator: 009 | Epoch: 001 | Batch: 000 | Loss: 14.30182 | Correct: 3/16


2022-04-01 19:07:58,011 - INFO: Epoch: 001 | Validation Acc: 20.833 % | Historical Best: 33.333 %


Estimator: 000 | Epoch: 002 | Batch: 000 | Loss: 5.83547 | Correct: 4/16
Estimator: 001 | Epoch: 002 | Batch: 000 | Loss: 1.66842 | Correct: 7/16
Estimator: 002 | Epoch: 002 | Batch: 000 | Loss: 3.92579 | Correct: 5/16
Estimator: 003 | Epoch: 002 | Batch: 000 | Loss: 3.68235 | Correct: 7/16
Estimator: 004 | Epoch: 002 | Batch: 000 | Loss: 3.64320 | Correct: 6/16
Estimator: 005 | Epoch: 002 | Batch: 000 | Loss: 6.55105 | Correct: 3/16
Estimator: 006 | Epoch: 002 | Batch: 000 | Loss: 4.56637 | Correct: 3/16
Estimator: 007 | Epoch: 002 | Batch: 000 | Loss: 2.35551 | Correct: 6/16
Estimator: 008 | Epoch: 002 | Batch: 000 | Loss: 2.14081 | Correct: 7/16
Estimator: 009 | Epoch: 002 | Batch: 000 | Loss: 6.30049 | Correct: 5/16


...

2022-04-01 19:13:17,555 - INFO: Epoch: 017 | Validation Acc: 50.000 % | Historical Best: 54.167 %


Estimator: 000 | Epoch: 018 | Batch: 000 | Loss: 0.10211 | Correct: 16/16
Estimator: 001 | Epoch: 018 | Batch: 000 | Loss: 0.05276 | Correct: 16/16
Estimator: 002 | Epoch: 018 | Batch: 000 | Loss: 0.15264 | Correct: 15/16
Estimator: 003 | Epoch: 018 | Batch: 000 | Loss: 0.15951 | Correct: 16/16
Estimator: 004 | Epoch: 018 | Batch: 000 | Loss: 0.07595 | Correct: 16/16
Estimator: 005 | Epoch: 018 | Batch: 000 | Loss: 0.18957 | Correct: 15/16
Estimator: 006 | Epoch: 018 | Batch: 000 | Loss: 0.24462 | Correct: 15/16
Estimator: 007 | Epoch: 018 | Batch: 000 | Loss: 0.06568 | Correct: 16/16
Estimator: 008 | Epoch: 018 | Batch: 000 | Loss: 0.52121 | Correct: 12/16
Estimator: 009 | Epoch: 018 | Batch: 000 | Loss: 0.53325 | Correct: 11/16


2022-04-01 19:13:31,708 - INFO: Epoch: 018 | Validation Acc: 50.000 % | Historical Best: 54.167 %


Estimator: 000 | Epoch: 019 | Batch: 000 | Loss: 0.06232 | Correct: 16/16
Estimator: 001 | Epoch: 019 | Batch: 000 | Loss: 0.13283 | Correct: 16/16
Estimator: 002 | Epoch: 019 | Batch: 000 | Loss: 0.10777 | Correct: 15/16
Estimator: 003 | Epoch: 019 | Batch: 000 | Loss: 0.14707 | Correct: 16/16
Estimator: 004 | Epoch: 019 | Batch: 000 | Loss: 0.11351 | Correct: 16/16
Estimator: 005 | Epoch: 019 | Batch: 000 | Loss: 0.24712 | Correct: 14/16
Estimator: 006 | Epoch: 019 | Batch: 000 | Loss: 0.44734 | Correct: 13/16
Estimator: 007 | Epoch: 019 | Batch: 000 | Loss: 0.06410 | Correct: 16/16
Estimator: 008 | Epoch: 019 | Batch: 000 | Loss: 0.10992 | Correct: 16/16
Estimator: 009 | Epoch: 019 | Batch: 000 | Loss: 0.53289 | Correct: 14/16


2022-04-01 19:13:45,779 - INFO: Epoch: 019 | Validation Acc: 54.167 % | Historical Best: 54.167 %


In [12]:
# Evaluating

accuracy = model.evaluate(test_loader)
print(f"Accuracy: {accuracy}")

Accuracy: 54.166666666666664


### Save and Reload

In [17]:
from torchensemble.utils import io

io.load(model, save_dir=src, logger=logger)  # reload


FileNotFoundError: [Errno 2] No such file or directory: '../../models/example_ensemble/VotingClassifier_MLP_10_ckpt'